<a href="https://colab.research.google.com/github/Dale274/GPT-3.5-Recipes-Generator-OpenAI-API-Key-/blob/main/GPT_3_5_Recipes_Generator_(OpenAI_API_Key).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install google-cloud-secret-manager
import openai
from google.cloud import secretmanager
from google.colab import auth, drive
import os
import json
import pandas as pd

# First, I authenticate and create a Secret Manager client
auth.authenticate_user()
client = secretmanager.SecretManagerServiceClient()

# Next, I input my Secret Manager project ID on Google Cloud to not expose my API in this code
project_id = "435552672817"
# I input the secret name used in the Secret Manager
secret_name = "OpenAI_API_Key"

# Access the secret version
name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
response = client.access_secret_version(request={"name": name})
api_key = response.payload.data.decode('UTF-8')

# I set my OpenAI API key
openai.api_key = api_key

# I mount my Google Drive
drive.mount('/content/drive')

# Change the working directory to the project folder
os.chdir('/content/drive/MyDrive/GPT-3 Recipes Generator/')

# Set the base path for the dataset folder
base_path = '/content/drive/MyDrive/GPT-3 Recipes Generator/Dataset/'

# Load the dataset from JSON files into a Pandas DataFrame
recipe_sources = ['ar', 'epi', 'fn']
df = pd.DataFrame()

sources, titles, ingredients, instructions = [], [], [], []

for recipe_source in recipe_sources:
    file_path = f'{base_path}recipes_raw_nosource_{recipe_source}.json'
    with open(file_path, 'r') as file:
        data = json.load(file)

    for _, recipe in data.items():
        sources.append(recipe_source)
        titles.append(recipe.get('title', ''))
        ingredients.append([ingredient.replace('ADVERTISEMENT', '') for ingredient in recipe.get('ingredients', [])])
        instructions.append(str(recipe.get('instructions', '')).replace('ADVERTISEMENT', '').replace('\n', ''))

df['source'] = sources
df['title'] = titles
df['ingredients'] = ingredients
df['instructions'] = instructions

# Preprocess the data
df['ingredient_count'] = df['ingredients'].str.len().fillna(0).astype(int)
df['instruction_length'] = df['instructions'].str.split().str.len().fillna(0).astype(int)

df = df[(df.ingredient_count >= 3) & (df.instruction_length >= 10)]

# Modify the original instructions
df['modified_instructions'] = df['instructions'].apply(lambda x: " ".join(x.split()[:5]))

df['combined'] = ' \n Ingredients: \n ' + df.ingredients.str.join(' \n ') + ' \n Instructions: \n' + df.modified_instructions

# I use the OpenAI API to generate recipe instructions
while True:
    prompt_text = input("Enter the list of ingredients (separated by commas or by spaces, whichever gets your recipe generated first!): ")
    prompt_text = 'Ingredients: ' + prompt_text.replace(',', '\n')

    print("Generating recipe instructions...")
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt_text + " \n Instructions: ",
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.7,
        top_p=0.95
    )

    instructions = response.choices[0].text.strip()

    print("Instructions:")
    print(instructions)

    choice = input("Do you want to continue? (y/n): ")
    if choice.lower() == "n":
      break
